#Import packages

In [1]:
from xml.dom import minidom
from sys import stdin
from urllib import request
from subprocess import call
import numpy as np
import pandas as pd
import itertools
import pyart



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



#initialize user-defined variables

In [2]:
date = "2020/01/01"
site = "KCRP"
bucketURL = "http://noaa-nexrad-level2.s3.amazonaws.com"
dirListURL = bucketURL+ "/?prefix=" + date + "/" + site

#list available files

In [3]:
def getText(nodelist):
    rc = []
    for node in nodelist:
        if node.nodeType == node.TEXT_NODE:
            rc.append(node.data)
    return ''.join(rc)

print ("listing files from %s" % dirListURL)

xmldoc = minidom.parse(request.urlopen(dirListURL))
itemlist = xmldoc.getElementsByTagName('Key')
print (len(itemlist) , "keys found...")

for x in itemlist:
    file = getText(x.childNodes)

listing files from http://noaa-nexrad-level2.s3.amazonaws.com/?prefix=2020/01/01/KCRP
257 keys found...


#read NEXRAD radar file -- having issue reading from http, temporarily reading from local directory

In [5]:
#https://s3.amazonaws.com/noaa-nexrad-level2/index.html#2020/01/01/KCRP/
#https://noaa-nexrad-level2.s3.amazonaws.com/2020/01/01/KCRP/KCRP20200101_000431_V06

#radar = pyart.io.read_nexrad_archive('https://noaa-nexrad-level2.s3.amazonaws.com/2020/01/01/KCRP/' + 'KCRP20200101_000431_V06')
#radar = pyart.io.read('https://noaa-nexrad-level2.s3.amazonaws.com/2020/01/01/KCRP/' + 'KCRP20200101_000431_V06')
radar = pyart.io.read('data/NEXRAD/KCRP20200101_000431_V06')


In [ ]:
#radar.info()

#confirm station location

In [6]:
altitude0 = radar.altitude['data'][0]
latitude0 = radar.latitude['data'][0]
longitude0 = radar.longitude['data'][0]
print(altitude0, latitude0, longitude0)

43.0 27.78401756286621 -97.5112533569336


#confirm number of rays and gates
#Ray attributes: azimuth, elevation, time
#Gate attributes: range
#Ray x Gate attributes: lat, lon, alt

In [7]:
print(radar.nrays)
print(radar.ngates)

6480
1832


#unfold lists of lists

In [8]:
gate_lat = radar.gate_latitude['data']
gate_lon = radar.gate_longitude['data']
gate_alt = radar.gate_altitude['data']
refl = radar.fields['reflectivity']['data']
velo = radar.fields['velocity']['data']

merged_lat = list(itertools.chain.from_iterable(gate_lat))
merged_lon = list(itertools.chain.from_iterable(gate_lon))
merged_alt = list(itertools.chain.from_iterable(gate_alt))
merged_refl = list(itertools.chain.from_iterable(refl))
merged_velo = list(itertools.chain.from_iterable(velo))

#expand time field

In [9]:
time_x1 =  radar.time['data']
time_xgates = [val for val in time_x1 for _ in range(radar.ngates)]

#visual checks -- make into real tests

In [10]:
print(radar.nrays * radar.ngates)
print(len(gate_lat))
print(len(gate_lat[0]))
print(len(radar.time['data']))
print(len(time_xgates))
print(len(merged_refl))

11871360
6480
1832
6480
11871360
11871360


#combine data into dataframe

In [ ]:
samples = pd.DataFrame(
    {'Gate latitude': merged_lat,
     'Gate longitude': merged_lon,
     'Gate altitude': merged_alt,
     'Gate time': time_xgates,
     'Reflectivity': merged_refl,
     'Velocity': merged_velo
    })
print(len(samples))
print(samples[0:50])